# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
# импортируем библитоеку pandas, прочитаем данные из таблицы data и присвоим значение таблицы переменной table 
import pandas as pd
table = pd.read_csv('/datasets/data.csv')

# выведем информаию о таблице и первые 20 строк для изучения таблицы
table.info()
table.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

1) В таблице 21525 строк и 12 колонок.

2) В колонках days_employed и total_income имеется 2 174 пропусков, клиенты о которых нет информации о статусе трудоустройства и не указан доход. Общее количество составляет почти 10%, следовательно данные лучше не удалять.

3) В колонке с образованием имеются разница в использовании регистра при вводе данных, данные придется обработать чтобы привести к одному виду.

4) в колонках days_employed и total_income необходимо будет обработать данные, так в текущем формате неудобно анализировать данные

## Шаг 2. Предобработка данных

### Обработка пропусков

Чтобы сохранить важные данные заменим пустые значения на средние в колонках total_income и days_employed.

In [2]:
# заполним пустые значения на средние значения в колонках и выведем результат 
table['days_employed'] = table['days_employed'].fillna(table['days_employed'].mean()) 
table['total_income'] = table['total_income'].fillna(table['total_income'].mean()) 
table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Теперь в таблице нет пустых значений можно переходить к дальнейшей обработке.
Пустые данные могли появиться из-за того что клиенты не указали информацию о доходе или его нет на данный момент.

### Замена типа данных

In [3]:
# Чтобы лишние цифры после запятой не мешали анализу поменяем тип данных в колонках total_income и days_employed 
table['total_income'] = table['total_income'].astype('int')
table['days_employed'] = table['days_employed'].astype('int')

# выведем информацию о таблице на экран
table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

После замены типов данных с ними будет проще работать. 
Как видно в выводе теперь в таблице нет данных с типом float.
Для замены использовался метод astype так как в колонках все значения были float и не нужно было получать информацию о возможных ошибках при изменении значений.

### Обработка дубликатов

In [4]:

# приведем в столбце education все значение к строчным
table['education'] = table['education'].str.lower()

# найдем дубликаты
print('количество дубликатов: ',table.duplicated().sum(),'\n')

# удалим дубликаты в таблице с данными и выведем информацию на экран
table = table.drop_duplicates().reset_index(drop=True)

# проверим новое количество строк
table.info()


количество дубликатов:  71 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Для удаления дубликатов использовался метод drop_duplicates без указания определенной колонки для удаления дубликатов с идентичными значениями во всех строчках, при удалении дубликатов также были сброшены индексы.

Дубликаты скорее всего появились из-за ошибок при вводе данных в таблицу - человеческий фактор.

### Лемматизация

In [5]:
# импортируем Mystem из библитеки pymystem3  и присовим переменной 'm' значение Mystem()
from pymystem3 import Mystem
m = Mystem()

#создадим цикл для лемматизации текста 
def lem_cycle (text):
    '''
    При обращениик функции, переменная text принимает переданное значение.
    Цикл перебирает значения переменной, лемматизирует значения переменной и возвращает лемматизированное значение.
    '''
    for i in text:
        return m.lemmatize(text)

# лемматизируем с помощью цункции lem_cycle колонку purpose создадим новую колонку с рузультатом лематизации
table['purpose_category'] = table['purpose'].apply(lem_cycle)

# попробуем сгруппировать данные по столбцу purpose_category чтобы просмотреть все уникальные значения
# воспользуемся конструкцией try-except,так как есть вероятность что код не сработает.
# (в процессе лематизации обрабатывались строки, следовательно в новой колонке дата фреймма purpose_category  тип данных значений это список. . 
try:
    table.groupby('purpose_category')['debt'].sum()
except:
    print('Type error / ошибка типа данных','\n')
    # Создадим две переменных. Первой присвоим значение колонки purpose_category , вторая же будет сохранять ункальные значения при переборе значений первой переменной в цикле.
    var_one = table['purpose_category']
    lem_unique = []
    for i in var_one:
        if i not in lem_unique:
            lem_unique.append(i)
    print(lem_unique)

Type error / ошибка типа данных 

[['покупка', ' ', 'жилье', '\n'], ['приобретение', ' ', 'автомобиль', '\n'], ['дополнительный', ' ', 'образование', '\n'], ['сыграть', ' ', 'свадьба', '\n'], ['операция', ' ', 'с', ' ', 'жилье', '\n'], ['образование', '\n'], ['на', ' ', 'проведение', ' ', 'свадьба', '\n'], ['покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n'], ['покупка', ' ', 'недвижимость', '\n'], ['покупка', ' ', 'коммерческий', ' ', 'недвижимость', '\n'], ['покупка', ' ', 'жилой', ' ', 'недвижимость', '\n'], ['строительство', ' ', 'собственный', ' ', 'недвижимость', '\n'], ['недвижимость', '\n'], ['строительство', ' ', 'недвижимость', '\n'], ['на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', '\n'], ['на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n'], ['операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '\n'], ['строительство', ' ', 'жилой', ' ', 'недвижимость', '\n'], ['жилье', '\n'], ['операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', '\n'],

**Вывод**

На основании лемматизации можно выделить 4 основные леммы:
1) свадьба
2) автомобиль
3) образование
4) недвижимость

### Категоризация данных

In [6]:
# Создадим цикл для категоризации данных в столбе purpose_category по 4 основным леммам
def category (input):
    if 'свадьба' in input:
        return 'свадьба'
    elif 'автомобиль' in input:
        return 'автомобиль'
    elif 'образование' in input:
        return 'образование'
    else:
        return 'недвижимость'

# Создадим цикл для категоризцаии данных по наличию/отсутвию долга
def credit_debt (input):
    if input == 0:
        return 'долга нет'
    else:
        return 'имеется долг'

# Создадим цикл для категоризцаии данных по наличии/отсутствию детей
def kids (input):
    if input == 0:
        return 'без детей'
    else:
        return 'есть дети'
        
# применим циклы с замещением занчение в столбце purpose_category и создание двух новых столбцов с информацией  о наличии детей и долга. 
table['purpose_category'] = table['purpose_category'].apply(category)
table['debt_status'] = table['debt'].apply(credit_debt)
table['kids_status'] = table['children'].apply(kids)

# выведем первые 10 строк таблицы на экран для проверки
table.head(10)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,debt_status,kids_status
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,долга нет,есть дети
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,долга нет,есть дети
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,долга нет,без детей
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,долга нет,есть дети
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,долга нет,без детей
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость,долга нет,без детей
6,0,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость,долга нет,без детей
7,0,-152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование,долга нет,без детей
8,2,-6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба,долга нет,есть дети
9,0,-2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость,долга нет,без детей


**Вывод**

Для проведения категоризации по столбцу purpose_category были выделены 4 основные леммы так ка они одна из них встречается в любом значении данного столбца.

Для других категоризаций было описано наличие или отсуттвие детей или долга для дальнейшей обработки информации и ответов на вопросы.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [7]:
# создадим новую таблицу и присвоим ей значение таблицы с информацие о клиентах
kids_kredit_check = table

#  добавим новую колонку со значеним статуса наличия детей и долга и сгруппируем по значению новой колнки для подсчета количества по каждой категории.
kids_kredit_check['category_kids_kredit'] = kids_kredit_check['kids_status']+', '+kids_kredit_check['debt_status']
kids_kredit_check = kids_kredit_check.groupby('category_kids_kredit')['debt'].count()

# выведем таблицу на экран
display(kids_kredit_check)

# проверим что все значения учтены в таблице
print(kids_kredit_check.sum()==table['debt'].count(),'\n')

# расчитаем процент отношения клиентов с долгами по каждой категории к общему количеству клиентов в категории
precent_withno_kids = kids_kredit_check.iloc[1]/(kids_kredit_check.iloc[0]+kids_kredit_check.iloc[1])
precent_with_kids = kids_kredit_check.iloc[3]/(kids_kredit_check.iloc[2]+kids_kredit_check.iloc[3])

# выведем значения на экран
print('Процент клиентов с детьми, которые имеют долг: {:.2%}'.format(precent_with_kids))
print('Процент клиентов без детей, которые имеют долг: {:.2%}'.format(precent_withno_kids))

category_kids_kredit
без детей, долга нет       13028
без детей, имеется долг     1063
есть дети, долга нет        6685
есть дети, имеется долг      678
Name: debt, dtype: int64

True 

Процент клиентов с детьми, которые имеют долг: 9.21%
Процент клиентов без детей, которые имеют долг: 7.54%


**Вывод**

По данным банка зависимость есть: клиенты с детьми на 1.7% чаще не возвращают кредит в срок.

Однако необходимо учитывать, что в данных имеется информация только о 21454 клиентах, вероятно, что при большей выборке процент клиентов, которые не возвращают кредит в срок может измениться в любую сторону.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [8]:
# создадим новую колонку и присвоим ей значение таблицы с информацие о клиентах(долг + семейный статус)
table['marriage_debt_status'] = table['family_status']+', '+table['debt_status']

# сгруппируем по значению новой колнки для подсчета количества по каждой категории.
marriage_check = table.groupby('marriage_debt_status')['debt'].count()

# выведем таблицу на экран
display(marriage_check)

# проверим что все значения учтены в таблице
print(marriage_check.sum()==table['debt'].count(),'\n')

# расчитаем процент отношения клиентов с долгами по каждой категории к общему количеству клиентов в категории
precent_not_married = marriage_check.iloc[1]/(marriage_check.iloc[0]+marriage_check.iloc[1])
precent_divorced = marriage_check.iloc[3]/(marriage_check.iloc[2]+marriage_check.iloc[3])
precent_widow = marriage_check.iloc[5]/(marriage_check.iloc[4]+marriage_check.iloc[5])
precent_civil_partners = marriage_check.iloc[7]/(marriage_check.iloc[6]+marriage_check.iloc[7])
precent_married = marriage_check.iloc[9]/(marriage_check.iloc[8]+marriage_check.iloc[9])

# выведем значения на экран
print('Процент клиентов в браке, которые имеют долг: {:.2%}'.format(precent_married))
print('Процент клиентов, которые состоят в гражданском браке и имеют долг: {:.2%}'.format(precent_civil_partners))
print('Процент разведенных клиентов, которые имеют долг: {:.2%}'.format(precent_divorced))
print('Процент овдовевших клиентов, которые имеют долг: {:.2%}'.format(precent_widow))
print('Процент клиентов, которые не состоят в браке и имеют долг: {:.2%}'.format(precent_not_married))

marriage_debt_status
Не женат / не замужем, долга нет        2536
Не женат / не замужем, имеется долг      274
в разводе, долга нет                    1110
в разводе, имеется долг                   85
вдовец / вдова, долга нет                896
вдовец / вдова, имеется долг              63
гражданский брак, долга нет             3763
гражданский брак, имеется долг           388
женат / замужем, долга нет             11408
женат / замужем, имеется долг            931
Name: debt, dtype: int64

True 

Процент клиентов в браке, которые имеют долг: 7.55%
Процент клиентов, которые состоят в гражданском браке и имеют долг: 9.35%
Процент разведенных клиентов, которые имеют долг: 7.11%
Процент овдовевших клиентов, которые имеют долг: 6.57%
Процент клиентов, которые не состоят в браке и имеют долг: 9.75%


**Вывод**

По данным банка наименьше процент должников это овдовевшие клиенты - 6,5%  должников.
Затем числятся разведенные клиенты - 7,1% должников
Только 7.55% клиентов в браке имеют долг.
Наибольший процент должников это клиенты которые не состоят в браке или имеют гражданский брак. 9,75% и 9,35% должников.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [9]:
# для определния уровня дохода необходимо ввести категоризацию
#  для этого посчитаем среднее значение и категоризируем данные с клиентами  средним/выше среднего дохода и ниже
average = table['total_income'].mean()

def income (input):
    if input < average:
        return 'доход ниже среднего'
    else:
        return 'доход выше среднего'

table['income_debt_status'] = table['total_income'].apply(income)

# присвоим колонке income_debt_status таблицы с информацией о клиентах (долг + уровень дохода)
table['income_debt_status'] = table['income_debt_status'] +', '+table['debt_status']

# сгруппируем по значению новой колнки для подсчета количества по каждой категории.
income_check = table.groupby('income_debt_status')['debt'].count()

# выведем таблицу на экран
display(income_check) 

# проверим что все значения учтены в таблице
print(income_check.sum()==table['debt'].count(),'\n')

# расчитаем процент отношения клиентов с долгами по каждой категории к общему количеству клиентов в категории
precent_above_avg= income_check.iloc[1]/(income_check.iloc[2]+income_check.iloc[3])
precent_below_avg= income_check.iloc[3]/(income_check.iloc[0]+income_check.iloc[1])

# выведем значения на экран
print('Процент клиентов с доходом ниже среднего, которые имеют долг: {:.2%}'.format(precent_below_avg))
print('Процент клиентов с доходом выше среднего, которые имеют долг: {:.2%}'.format(precent_above_avg))

income_debt_status
доход выше среднего, долга нет        8824
доход выше среднего, имеется долг      738
доход ниже среднего, долга нет       10889
доход ниже среднего, имеется долг     1003
Name: debt, dtype: int64

True 

Процент клиентов с доходом ниже среднего, которые имеют долг: 10.49%
Процент клиентов с доходом выше среднего, которые имеют долг: 6.21%


**Вывод**

Клиенты с доходом ниже среднего чаще задерживают выплату кредит чем клиенты с доходом выше среднего (10% против 6%)

- Как разные цели кредита влияют на его возврат в срок?

In [10]:
# присвоим колонке purpose_debt_status таблицы с информацией о клиентах (долг + категория цели кредита)
table['purpose_debt_status'] = table['purpose_category']+', '+table['debt_status']

# # сгруппируем по значению новой колнки для подсчета количества по каждой категории.
purpose_check = table.groupby('purpose_debt_status')['debt'].count()

# выведем таблицу на экран
display(purpose_check)

# проверим что все значения учтены в таблице
print(purpose_check.sum()==table['debt'].count(),'\n')

# расчитаем процент отношения клиентов с долгами по каждой категории к общему количеству клиентов в категории
precent_auto = purpose_check.iloc[1]/(purpose_check.iloc[0]+purpose_check.iloc[1])
precent_realty = purpose_check.iloc[3]/(purpose_check.iloc[2]+purpose_check.iloc[3]) 
precent_education_= purpose_check.iloc[5]/(purpose_check.iloc[4]+purpose_check.iloc[5])
precent_marriage = purpose_check.iloc[7]/(purpose_check.iloc[6]+purpose_check.iloc[7])

# выведем значения на экран
print('Процент клиентов взявшие кредит на автомобиль, которые имеют долг: {:.2%}'.format(precent_auto))
print('Процент клиентов взявшие кредит на недвижимость, которые имеют долг: {:.2%}'.format(precent_realty))
print('Процент клиентов взявшие кредит на образование, которые имеют долг: {:.2%}'.format(precent_education_))
print('Процент клиентов взявшие кредит на свадьбу, которые  имеют долг: {:.2%}'.format(precent_marriage))

purpose_debt_status
автомобиль, долга нет          3903
автомобиль, имеется долг        403
недвижимость, долга нет       10029
недвижимость, имеется долг      782
образование, долга нет         3643
образование, имеется долг       370
свадьба, долга нет             2138
свадьба, имеется долг           186
Name: debt, dtype: int64

True 

Процент клиентов взявшие кредит на автомобиль, которые имеют долг: 9.36%
Процент клиентов взявшие кредит на недвижимость, которые имеют долг: 7.23%
Процент клиентов взявшие кредит на образование, которые имеют долг: 9.22%
Процент клиентов взявшие кредит на свадьбу, которые  имеют долг: 8.00%


**Вывод**

Только 7% клиентов целью кредита которых была покупка недвижимости не вернули кредит в срок.
8% клиентов которые брали кредит на свадьбу не вернули его в срок.
Наибольшее количество клиентов не вернувших кредит в срок брали кредит на автомобиль и образование (9,3% и 9,2%

## Шаг 4. Общий вывод

В процессе анализа данных было сделано:
1.  определины и заполнены пропущенные значения;
2.  заменен вещественный тип данных на целочисленный:
3.  удалены дубликаты:
4.	выделены леммы в значениях столбца с целями получения кредита:
5.	категоризированны данные:
6.  были даны ответы на все поставленные вопросы и показано как влияют различные характеристики клиентов и целей кредита на его возврат в срок.


## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.